In [4]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models import resnet18

import time
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

C:\Users\Adam\AppData\Local\Temp\ipykernel_18736\1802083502.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Prepare transformers and DataLoaders

In [5]:
transformTrainAug = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transformTest = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


trainData = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transformTest)
trainLoader = torch.utils.data.DataLoader(trainData, batch_size=64,shuffle=True, num_workers=2)

trainDataAug =  torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transformTrainAug)
trainLoaderAug = torch.utils.data.DataLoader(trainDataAug, batch_size=64,shuffle=True, num_workers=2)

testData = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transformTest)
testLoader = torch.utils.data.DataLoader(testData, batch_size=64, shuffle=False, num_workers=2)

In [19]:
def trainModel(nnModel, trainLoader, optimizerMethod=optim.SGD, lossFunction=nn.CrossEntropyLoss(), lRate = 0.005, n_epochs = 10):
    optimizer = optimizerMethod(nnModel.parameters(), lRate, momentum=0.5)

    start = time.time()
    for epch in range(n_epochs): 
        # before each epoch DataLoader reshuffles 
        for idx,dataBatch in enumerate(trainLoader):
            input, labels = dataBatch
            
            # zeroing out the gradient
            optimizer.zero_grad()

            # forward + backward + optimize
            output = nnModel(input)
            loss = lossFunction(output, labels)
            loss.backward()
            optimizer.step()

            if idx % 100 == 0:
                print(f"Loss: {loss.item():.3f}  Epoch:{epch} Iteration:{idx}")
    
    end = time.time()
    return nnModel, end - start

def evalNN(nnModel, testLoader):
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            outputs = nnModel(images)
            _, predicted = torch.max(outputs.data, 1)
            y_true += labels.cpu().numpy().tolist()
            y_pred += predicted.cpu().numpy().tolist()

        return y_true, y_pred
    
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Training Lenet without data augemented

In [12]:
trainedNet, meausured_time  = trainModel(LeNet(), trainLoader)
label_true, label_pred = evalNN(trainedNet, testLoader)
pd.DataFrame( [{"Accuracy":accuracy_score(label_true, label_pred),
                "Precision":precision_score(label_true, label_pred,average="macro"),
                "Trainig time":meausured_time}])

Loss: 2.298  Epoch:0 Iteration:0
Loss: 2.299  Epoch:0 Iteration:100
Loss: 2.294  Epoch:0 Iteration:200
Loss: 2.280  Epoch:0 Iteration:300
Loss: 2.271  Epoch:0 Iteration:400
Loss: 2.221  Epoch:0 Iteration:500
Loss: 2.106  Epoch:0 Iteration:600
Loss: 1.835  Epoch:0 Iteration:700
Loss: 1.286  Epoch:0 Iteration:800
Loss: 0.936  Epoch:0 Iteration:900
Loss: 0.640  Epoch:1 Iteration:0
Loss: 0.703  Epoch:1 Iteration:100
Loss: 0.577  Epoch:1 Iteration:200
Loss: 0.586  Epoch:1 Iteration:300
Loss: 0.889  Epoch:1 Iteration:400
Loss: 0.760  Epoch:1 Iteration:500
Loss: 0.542  Epoch:1 Iteration:600
Loss: 0.468  Epoch:1 Iteration:700
Loss: 0.492  Epoch:1 Iteration:800
Loss: 0.421  Epoch:1 Iteration:900
Loss: 0.480  Epoch:2 Iteration:0
Loss: 0.484  Epoch:2 Iteration:100
Loss: 0.433  Epoch:2 Iteration:200
Loss: 0.580  Epoch:2 Iteration:300
Loss: 0.509  Epoch:2 Iteration:400
Loss: 0.483  Epoch:2 Iteration:500
Loss: 0.381  Epoch:2 Iteration:600
Loss: 0.514  Epoch:2 Iteration:700
Loss: 0.525  Epoch:2 Itera

,Accuracy,Precision,Trainig time
0,0.849,0.863283,94.128565


### Training resnet with data augemented

In [10]:
trainedNetAug, meausured_timeAug  = trainModel(LeNet(), trainLoaderAug)
label_true, label_pred = evalNN(trainedNetAug, testLoader)
pd.DataFrame( [{"Accuracy":accuracy_score(label_true, label_pred),
                "Precision":precision_score(label_true, label_pred,average="macro"),
                "Trainig time":meausured_timeAug}])

Loss: 2.303  Epoch:0 Iteration:0
Loss: 2.312  Epoch:0 Iteration:100
Loss: 2.299  Epoch:0 Iteration:200
Loss: 2.297  Epoch:0 Iteration:300
Loss: 2.285  Epoch:0 Iteration:400
Loss: 2.277  Epoch:0 Iteration:500
Loss: 2.247  Epoch:0 Iteration:600
Loss: 2.210  Epoch:0 Iteration:700
Loss: 2.093  Epoch:0 Iteration:800
Loss: 1.759  Epoch:0 Iteration:900
Loss: 1.548  Epoch:1 Iteration:0
Loss: 1.410  Epoch:1 Iteration:100
Loss: 1.440  Epoch:1 Iteration:200
Loss: 1.243  Epoch:1 Iteration:300
Loss: 1.128  Epoch:1 Iteration:400
Loss: 0.895  Epoch:1 Iteration:500
Loss: 1.011  Epoch:1 Iteration:600
Loss: 0.992  Epoch:1 Iteration:700
Loss: 1.032  Epoch:1 Iteration:800
Loss: 0.918  Epoch:1 Iteration:900
Loss: 0.993  Epoch:2 Iteration:0
Loss: 0.801  Epoch:2 Iteration:100
Loss: 0.714  Epoch:2 Iteration:200
Loss: 0.727  Epoch:2 Iteration:300
Loss: 0.874  Epoch:2 Iteration:400
Loss: 0.782  Epoch:2 Iteration:500
Loss: 0.864  Epoch:2 Iteration:600
Loss: 0.702  Epoch:2 Iteration:700
Loss: 0.614  Epoch:2 Itera

,Accuracy,Precision,Trainig time
0,0.7987,0.798838,128.270577


### 3 layer Lenet with class

In [20]:
class LeNet_MoreConv(nn.Module):
  def __init__(self):
      super(LeNet_MoreConv, self).__init__()
      self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
      self.pool1 = nn.MaxPool2d(kernel_size=2)
      self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
      self.pool2 = nn.MaxPool2d(kernel_size=2)
      self.conv3 = nn.Conv2d(16, 32, kernel_size=3)  # Added third convolutional layer
      self.pool3 = nn.MaxPool2d(kernel_size=2)
      self.fc1 = nn.Linear(32 * 2 * 2, 120)  # Adjusted input size for fc1
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool1(nn.functional.relu(self.conv1(x)))
      x = self.pool2(nn.functional.relu(self.conv2(x)))
      x = self.pool3(nn.functional.relu(self.conv3(x)))  # Added third pooling layer
      x = x.view(-1, 32 * 2 * 2)  # Reshaped for fc1
      x = nn.functional.relu(self.fc1(x))
      x = nn.functional.relu(self.fc2(x))
      x = self.fc3(x)
      return x

### 3-layered lenet model without data augementation

In [21]:
trainedNet, meausured_time  = trainModel(LeNet3(), trainLoader)
label_true, label_pred = evalNN(trainedNet, testLoader)
pd.DataFrame( [{"Accuracy":accuracy_score(label_true, label_pred),
                "Precision":precision_score(label_true, label_pred,average="macro"),
                "Trainig time":meausured_time}])

ValueError: Expected input batch_size (16) to match target batch_size (64).

### 3-layered lenet model with data augmentation

In [18]:
trainedNet, meausured_time  = trainModel(LeNet3(), trainLoaderAug)
label_true, label_pred = evalNN(trainedNet, testLoader)
pd.DataFrame( [{"Accuracy":accuracy_score(label_true, label_pred),
                "Precision":precision_score(label_true, label_pred,average="macro"),
                "Trainig time":meausured_time}])

RuntimeError: Calculated padded input size per channel: (4 x 4). Kernel size: (5 x 5). Kernel size can't be greater than actual input size